In [1]:
#Liberías para tablas
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",1000)
pd.set_option("display.max_rows",1000)

# ----------------------------------------------------------------------------------------------------------

#Librerías gráficas
import matplotlib.pyplot as plt
plt.style.use("ggplot") # to make matplotlib look better
%matplotlib inline
import seaborn as sns
import plotly
from plotly import graph_objs as go
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)
BKCF_colors = ['#e32592','#fae700','#34b233','#58585a','#939597','#E0E1DD','#5bcbf5','#fe4540','#c7be71'] # to use with cufflinks
layout = go.Layout(colorway=BKCF_colors) # to use with plain plotly
import cufflinks as cf
cf.set_config_file(offline=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) # to allow plotly offline for jupyter notebooks
import pandas_bokeh
pandas_bokeh.output_notebook()
from bokeh.plotting import save # to allow saving pandas bokeh plots into html files
from IPython.core.display import display, HTML
from plotly.subplots import make_subplots
# ----------------------------------------------------------------------------------------------------------

#Dash
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

#Ipywidgets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

#Otras
import yfinance as yf
from datetime import date
import calendar
import pickle
import sys,os
from varname import nameof
from scipy import stats
import operator
import calendar
from scipy.stats import norm
from scipy import stats
#import statsmodels.api as sm
import pylab
from ipywidgets import widgets
import warnings
warnings.filterwarnings('ignore')
import datetime
import os

C:\Users\Javier\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Loading BokehJS ...

In [2]:
#Save in pickle format
def save_obj(obj, name, path="./obj"):
    if not(os.path.isdir(path)):
        os.mkdir(path)
        print("Creado directorio './obj' donde guardar objeto")
    with open("{}/{}.pkl".format(path,name), "wb") as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
#Load a pickle        
def load_obj(name, path="./obj"):
    with open("{}/{}.pkl".format(path, name), "rb") as f:
        return pickle.load(f)
    
#Download json
def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

def plot_metrica(df, metrica):
    df = df[['symbol','date','{0}'.format(metrica)]]
    colors = ['orange', 'pink','purple','brown','blue','green','black','grey','yellow','azure','gold']
    symbols = df['symbol'].drop_duplicates().values.tolist()
    if len(symbols)>12:
        print("Introduce como máximo 12 compañías")
    else:
        fig = px.line(df[df['symbol']==symbols[0]], x = df.columns[1], y= df.columns[2], color = 'symbol',
                      color_discrete_sequence=['red'],#line_dash_sequence=['dash'], 
                      title="{0}".format(metrica))
        for i in range(1,len(symbols)):
            fig1 = px.line(df[df['symbol']==symbols[i]], x = df.columns[1], y = df.columns[2], color = 'symbol',
                           color_discrete_sequence=[colors[i-1]])
            fig.add_trace(fig1.data[0])
        return fig
    
from ipywidgets import interact, IntSlider
from IPython.display import display

def freeze_header(df, num_rows=300, num_columns=10, step_rows=1,step_columns=1):
    """
    Freeze the headers (column and index names) of a Pandas DataFrame. A widget
    enables to slide through the rows and columns.

    Parameters
    ----------
    df : Pandas DataFrame
        DataFrame to display
    num_rows : int, optional
        Number of rows to display
    num_columns : int, optional
        Number of columns to display
    step_rows : int, optional
        Step in the rows
    step_columns : int, optional
        Step in the columns

    Returns
    -------
    Displays the DataFrame with the widget
    """
    @interact(last_row=IntSlider(min=min(num_rows, df.shape[0]),
                                 max=df.shape[0],
                                 step=step_rows,
                                 description='rows',
                                 readout=False,
                                 disabled=False,
                                 continuous_update=True,
                                 orientation='horizontal',
                                 slider_color='purple'),
              last_column=IntSlider(min=min(num_columns, df.shape[1]),
                                    max=df.shape[1],
                                    step=step_columns,
                                    description='columns',
                                    readout=False,
                                    disabled=False,
                                    continuous_update=True,
                                    orientation='horizontal',
                                    slider_color='purple'))
    def _freeze_header(last_row, last_column):
        display(df.iloc[max(0, last_row-num_rows):last_row,
                        max(0, last_column-num_columns):last_column])

# Financial data

In [54]:
%%time
#########################Financial data######################################
financial_data = load_obj("financial_data")
financial_data = financial_data[(financial_data['dimension']=='ARY')|(financial_data['dimension']=='ARQ')]

#######################Tickers##########################################
tickers = pd.read_csv("./data_sharadar/tickers.csv")
tickers = tickers[['ticker','name','exchange','isdelisted','category',
                  'sicsector','sicindustry','famasector','famaindustry','sector','industry',
                  'scalemarketcap','scalerevenue','location',
                  'lastupdated','firstadded','firstpricedate','lastpricedate',
                  'firstquarter','lastquarter','secfilings','companysite']]
financial_data = financial_data.merge(tickers)

########################Indicators############################################
indicators = pd.read_csv("./data_sharadar/indicators.csv")
dict_indicators = dict(zip(indicators.indicator, indicators.title))

########################Rename the indicators and create two datasets###########################
financial_data  = financial_data.rename(columns=dict_indicators)

########################Lets create two different datasets######################################
financial_data_quarter = financial_data[financial_data['Dimension']=="ARQ"].drop_duplicates(['Ticker Symbol','Calendar Date']).rename(columns={"Issuer Name":"Company"})
save_obj(financial_data_quarter,"financial_data_quarter")

financial_data_year = financial_data[financial_data['Dimension']=="ARY"].drop_duplicates(['Ticker Symbol','Calendar Date']).rename(columns={"Issuer Name":"Company"})
save_obj(financial_data_year,"financial_data_year")

Wall time: 27.1 s


sp_500 = pd.read_csv("./data_sharadar/sp_500.csv")
daily = pd.read_csv("./data_sharadar/daily.csv")
sp_500[sp_500['ticker'] =='FB']
daily[daily['ticker']=='VRSN'].sort_values(by='date')
#freeze_header(financial_data[(financial_data['ticker']=="SVNX")&(financial_data['dimension']=="ARY")],num_columns=15,num_rows=12)

# Análisis

In [82]:
%%time
financial_data_quarter_all = load_obj("financial_data_quarter")
financial_data_quarter_all = financial_data_quarter_all.replace(' ', '_', regex=True)
display(financial_data_quarter_all.head())

financial_data_year_all = load_obj("financial_data_year")
financial_data_year_all = financial_data_year_all.replace(' ', '_', regex=True)
financial_data_year_all.head()

,Ticker Symbol,Dimension,Calendar Date,Date Key,Report Period,Last Updated Date,Accumulated Other Comprehensive Income,Total Assets,Average Assets,Current Assets,Assets Non-Current,Asset Turnover,Book Value per Share,Capital Expenditure,Cash and Equivalents,Cash and Equivalents (USD),Cost of Revenue,Consolidated Income,Current Ratio,Debt to Equity Ratio,Total Debt,Debt Current,Debt Non-Current,Total Debt (USD),Deferred Revenue,Depreciation Amortization & Accretion,Deposit Liabilities,Dividend Yield,Dividends per Basic Common Share,Earning Before Interest & Taxes (EBIT),Earnings Before Interest Taxes & Depreciation Amortization (EBITDA),EBITDA Margin,Earnings Before Interest Taxes & Depreciation Amortization (USD),Earning Before Interest & Taxes (USD),Earnings before Tax,Earnings per Basic Share,Earnings per Diluted Share,Earnings per Basic Share (USD),Shareholders Equity,Average Equity,Shareholders Equity (USD),Enterprise Value - Daily,Enterprise Value over EBIT - Daily,Enterprise Value over EBITDA - Daily,Free Cash Flow,Free Cash Flow per Share,Foreign Currency to USD Exchange Rate,Gross Profit,Gross Margin,Goodwill and Intangible Assets,Interest Expense,Invested Capital,Invested Capital Average,Inventory,Investments,Investments Current,Investments Non-Current,Total Liabilities,Current Liabilities,Liabilities Non-Current,Market Capitalization - Daily,Net Cash Flow / Change in Cash & Cash Equivalents,Net Cash Flow - Business Acquisitions and Disposals,Issuance (Purchase) of Equity Shares,Issuance (Repayment) of Debt Securities,Payment of Dividends & Other Cash Distributions,Net Cash Flow from Financing,Net Cash Flow from Investing,Net Cash Flow - Investment Acquisitions and Disposals,Net Cash Flow from Operations,Effect of Exchange Rate Changes on Cash,Net Income,Net Income Common Stock,Net Income Common Stock (USD),Net Loss Income from Discontinued Operations,Net Income to Non-Controlling Interests,Profit Margin,Operating Expenses,Operating Income,Trade and Non-Trade Payables,Payout Ratio,Price to Book Value - Daily,Price Earnings (Damodaran Method) - Daily,Price to Earnings Ratio,Property Plant & Equipment Net,Preferred Dividends Income Statement Impact,Price,Price Sales (Damodaran Method) - Daily,Price to Sales Ratio,Trade and Non-Trade Receivables,Accumulated Retained Earnings (Deficit),Revenues,Revenues (USD),Research and Development Expense,Return on Average Assets,Return on Average Equity,Return on Invested Capital,Return on Sales,Share Based Compensation,Selling General and Administrative Expense,Share Factor,Shares (Basic),Weighted Average Shares,Weighted Average Shares Diluted,Sales per Share,Tangible Asset Value,Tax Assets,Income Tax Expense,Tax Liabilities,Tangible Assets Book Value per Share,Working Capital,Company,Stock Exchange,Is Delisted?,Issuer Category,SIC Sector,SIC Industry,Fama Sector,Fama Industry,Sector,Industry,Company Scale - Market Cap,Company Scale - Revenue,Location,First Added Date,First Price Date,Last Price Date,First Quarter,Last Quarter,SEC Filings URL,Company Website URL
0,A,ARQ,1999-12-31,2000-03-15,2000-01-31,2020-03-03,53000000.0,7.107000e+09,NaN,4.982000e+09,2.125000e+09,NaN,10.219,-30000000.0,1.368000e+09,1.368000e+09,1.160000e+09,131000000.0,2.410,0.584,665000000.0,111000000.0,554000000.0,665000000.0,281000000.0,96000000.0,0.0,0.0,0.0,202000000.0,298000000.0,0.133,298000000.0,202000000.0,202000000.0,0.30,0.30,0.30,4.486000e+09,NaN,4.486000e+09,5.096060e+10,NaN,NaN,354000000.0,0.806,1.0,1.086000e+09,0.484,0.000000e+00,0.0,4.337000e+09,NaN,1.567000e+09,42000000.0,42000000.0,0.0,2.621000e+09,2.067000e+09,554000000.0,5.166360e+10,1.368000e+09,-160000000.0,2.068000e+09,111000000.0,0.0,1.192000e+09,-208000000.0,-42000000.0,384000000.0,0.0,131000000.0,131000000.0,131000000.0,0.0,0.0,0.058,9.150000e+08,171000000.0,635000000.0,0.0,11.517,NaN,NaN,1.408000e+09,0.0,114.30,NaN,NaN,1.445000e+09,131000000.0,2.246000e+09,2.246000e+09,290000000.0,NaN,NaN,NaN,NaN,0.0,625000000.0,1.0,452000000.0,439000000

Wall time: 30.2 s


,Ticker Symbol,Dimension,Calendar Date,Date Key,Report Period,Last Updated Date,Accumulated Other Comprehensive Income,Total Assets,Average Assets,Current Assets,Assets Non-Current,Asset Turnover,Book Value per Share,Capital Expenditure,Cash and Equivalents,Cash and Equivalents (USD),Cost of Revenue,Consolidated Income,Current Ratio,Debt to Equity Ratio,Total Debt,Debt Current,Debt Non-Current,Total Debt (USD),Deferred Revenue,Depreciation Amortization & Accretion,Deposit Liabilities,Dividend Yield,Dividends per Basic Common Share,Earning Before Interest & Taxes (EBIT),Earnings Before Interest Taxes & Depreciation Amortization (EBITDA),EBITDA Margin,Earnings Before Interest Taxes & Depreciation Amortization (USD),Earning Before Interest & Taxes (USD),Earnings before Tax,Earnings per Basic Share,Earnings per Diluted Share,Earnings per Basic Share (USD),Shareholders Equity,Average Equity,Shareholders Equity (USD),Enterprise Value - Daily,Enterprise Value over EBIT - Daily,Enterprise Value over EBITDA - Daily,Free Cash Flow,Free Cash Flow per Share,Foreign Currency to USD Exchange Rate,Gross Profit,Gross Margin,Goodwill and Intangible Assets,Interest Expense,Invested Capital,Invested Capital Average,Inventory,Investments,Investments Current,Investments Non-Current,Total Liabilities,Current Liabilities,Liabilities Non-Current,Market Capitalization - Daily,Net Cash Flow / Change in Cash & Cash Equivalents,Net Cash Flow - Business Acquisitions and Disposals,Issuance (Purchase) of Equity Shares,Issuance (Repayment) of Debt Securities,Payment of Dividends & Other Cash Distributions,Net Cash Flow from Financing,Net Cash Flow from Investing,Net Cash Flow - Investment Acquisitions and Disposals,Net Cash Flow from Operations,Effect of Exchange Rate Changes on Cash,Net Income,Net Income Common Stock,Net Income Common Stock (USD),Net Loss Income from Discontinued Operations,Net Income to Non-Controlling Interests,Profit Margin,Operating Expenses,Operating Income,Trade and Non-Trade Payables,Payout Ratio,Price to Book Value - Daily,Price Earnings (Damodaran Method) - Daily,Price to Earnings Ratio,Property Plant & Equipment Net,Preferred Dividends Income Statement Impact,Price,Price Sales (Damodaran Method) - Daily,Price to Sales Ratio,Trade and Non-Trade Receivables,Accumulated Retained Earnings (Deficit),Revenues,Revenues (USD),Research and Development Expense,Return on Average Assets,Return on Average Equity,Return on Invested Capital,Return on Sales,Share Based Compensation,Selling General and Administrative Expense,Share Factor,Shares (Basic),Weighted Average Shares,Weighted Average Shares Diluted,Sales per Share,Tangible Asset Value,Tax Assets,Income Tax Expense,Tax Liabilities,Tangible Assets Book Value per Share,Working Capital,Company,Stock Exchange,Is Delisted?,Issuer Category,SIC Sector,SIC Industry,Fama Sector,Fama Industry,Sector,Industry,Company Scale - Market Cap,Company Scale - Revenue,Location,First Added Date,First Price Date,Last Price Date,First Quarter,Last Quarter,SEC Filings URL,Company Website URL
81,A,ARY,1999-12-31,2000-01-25,1999-10-31,2020-03-03,0.0,5.444000e+09,NaN,3.538000e+09,1.906000e+09,NaN,8.900,-360000000.0,0.000000e+00,0.000000e+00,4.388000e+09,5.120000e+08,2.105,0.610,0.000000e+00,0.0,0.000000e+00,0.000000e+00,241000000.0,475000000.0,0.0,0.0,0.0,7.870000e+08,1.262000e+09,0.151,1.262000e+09,7.870000e+08,7.870000e+08,1.35,1.35,1.35,3.382000e+09,NaN,3.382000e+09,3.059588e+10,39.0,24.244,101000000.0,0.266,1.0,3.943000e+09,0.473,0.000000e+00,0.0,3.763000e+09,NaN,1.499000e+09,0.0,0.0,0.0,2.062000e+09,1.681000e+09,3.810000e+08,3.059588e+10,0.0,45000000.0,0.000000e+00,0.000000e+00,0.0,-1.520000e+08,-3.090000e+08,0.0,461000000.0,NaN,5.120000e+08,5.120000e+08,5.120000e+08,0.0,0.0,0.061,3.202000e+09,7.410000e+08,510000000.0,0.0,9.047,59.758,50.141,1.387000e+09,0.0,67.69,3.673,3.088,1.635000e+09,NaN,8.331000e+09,8.331000e+09,9.970000e+08,NaN,NaN,NaN,0.094,0.0,2.205000e+09,1.0,452000000.0,380000000.0,380000000.0,21.924

In [83]:
#Parameters(asegurarse de que son días entre semana)
start_date = '1980-01-02' #Formato año-mes-dia: 3 de enero de 2013 #print(calendar.weekday(2013, 1, 2)) #Formato año,mes,dia
final_date = '2020-05-31' #Formato año-mes-dia: 3 de enero de 2020 #print(calendar.weekday(2020, 1, 31))
company_tickers = ["MSFT","DIS","KO","MCD"]
company_weighs = [0.3,0.3,0.2,0.2]
initial_investment = 10000
benchmark_tickers = ['^GSPC','^IXIC']
benchmark_names = ['sp500','nasdaq']

###############Company names##################
company_names = []
for company in company_tickers:
    company_names.append(financial_data_year_all[financial_data_year_all['Ticker Symbol']==company]['Company'].values[0])
company_names.append('total')
company_names

financial_data_year = financial_data_year_all[financial_data_year_all['Ticker Symbol'].isin(company_tickers)]
financial_data_year.sort_values(by="Calendar Date").head(10)

,Ticker Symbol,Dimension,Calendar Date,Date Key,Report Period,Last Updated Date,Accumulated Other Comprehensive Income,Total Assets,Average Assets,Current Assets,Assets Non-Current,Asset Turnover,Book Value per Share,Capital Expenditure,Cash and Equivalents,Cash and Equivalents (USD),Cost of Revenue,Consolidated Income,Current Ratio,Debt to Equity Ratio,Total Debt,Debt Current,Debt Non-Current,Total Debt (USD),Deferred Revenue,Depreciation Amortization & Accretion,Deposit Liabilities,Dividend Yield,Dividends per Basic Common Share,Earning Before Interest & Taxes (EBIT),Earnings Before Interest Taxes & Depreciation Amortization (EBITDA),EBITDA Margin,Earnings Before Interest Taxes & Depreciation Amortization (USD),Earning Before Interest & Taxes (USD),Earnings before Tax,Earnings per Basic Share,Earnings per Diluted Share,Earnings per Basic Share (USD),Shareholders Equity,Average Equity,Shareholders Equity (USD),Enterprise Value - Daily,Enterprise Value over EBIT - Daily,Enterprise Value over EBITDA - Daily,Free Cash Flow,Free Cash Flow per Share,Foreign Currency to USD Exchange Rate,Gross Profit,Gross Margin,Goodwill and Intangible Assets,Interest Expense,Invested Capital,Invested Capital Average,Inventory,Investments,Investments Current,Investments Non-Current,Total Liabilities,Current Liabilities,Liabilities Non-Current,Market Capitalization - Daily,Net Cash Flow / Change in Cash & Cash Equivalents,Net Cash Flow - Business Acquisitions and Disposals,Issuance (Purchase) of Equity Shares,Issuance (Repayment) of Debt Securities,Payment of Dividends & Other Cash Distributions,Net Cash Flow from Financing,Net Cash Flow from Investing,Net Cash Flow - Investment Acquisitions and Disposals,Net Cash Flow from Operations,Effect of Exchange Rate Changes on Cash,Net Income,Net Income Common Stock,Net Income Common Stock (USD),Net Loss Income from Discontinued Operations,Net Income to Non-Controlling Interests,Profit Margin,Operating Expenses,Operating Income,Trade and Non-Trade Payables,Payout Ratio,Price to Book Value - Daily,Price Earnings (Damodaran Method) - Daily,Price to Earnings Ratio,Property Plant & Equipment Net,Preferred Dividends Income Statement Impact,Price,Price Sales (Damodaran Method) - Daily,Price to Sales Ratio,Trade and Non-Trade Receivables,Accumulated Retained Earnings (Deficit),Revenues,Revenues (USD),Research and Development Expense,Return on Average Assets,Return on Average Equity,Return on Invested Capital,Return on Sales,Share Based Compensation,Selling General and Administrative Expense,Share Factor,Shares (Basic),Weighted Average Shares,Weighted Average Shares Diluted,Sales per Share,Tangible Asset Value,Tax Assets,Income Tax Expense,Tax Liabilities,Tangible Assets Book Value per Share,Working Capital,Company,Stock Exchange,Is Delisted?,Issuer Category,SIC Sector,SIC Industry,Fama Sector,Fama Industry,Sector,Industry,Company Scale - Market Cap,Company Scale - Revenue,Location,First Added Date,First Price Date,Last Price Date,First Quarter,Last Quarter,SEC Filings URL,Company Website URL
202087,DIS,ARY,1993-12-31,1993-12-22,1993-09-30,2020-02-05,36700000.0,1.175110e+10,NaN,NaN,NaN,NaN,3.080,7.937000e+08,3.630000e+08,3.630000e+08,6.804700e+09,2.998000e+08,NaN,1.336,2.385800e+09,NaN,NaN,2.385800e+09,0.000000e+00,3.642000e+08,0.0,0.005,0.080,8.302600e+08,1.194460e+09,0.140,1.194460e+09,8.302600e+08,6.725600e+08,0.183,0.183,0.183,5.030500e+09,NaN,5.030500e+09,2.589854e+10,31.0,21.682,2.938900e+09,1.799,1.0,1.724500e+09,0.202,0.000000e+00,157700000.0,1.377390e+10,NaN,608900000.0,1.888500e+09,NaN,NaN,6.720600e+09,NaN,NaN,2.387574e+10,-4.018000e+08,1.401000e+08,-3.160000e+07,1.368000e+08,-128600000.0,1.127000e+08,2.659700e+09,4.815000e+08,2.145200e+09,0.0,2.998000e+08,2.998000e+08,2.998000e+08,0.0,0.0,0.035,1.642000e+08,1.560300e+09,2.530100e+09,0.437,4.746,79.639,81.056,5.228200e+09,0.0,14.833,2.799,2.841,1.390300e+09,4.833100e+09,8.529200e+09,8.529200e+09,0.000000e+00,NaN,NaN,NaN,0.097,0.0,1.642000e+08,1.0,1.60960

## Revenue

In [84]:
order_list = financial_data_year.sort_values(by="Company",ascending=True)['Company'].unique().tolist()
category_orders={'Company': order_list}

px.bar(financial_data_year, x="Calendar Date", y="Revenues", #barmode='group',
       facet_col = "Company",color="Company",title="Revenues", category_orders = category_orders).show()